In [2]:
#!/usr/bin/env python
'''Project: Towards the development of a simple fingerprint representation for
the efficient comparison and clustering of large datasets of RNA sequences
By: Abdulvahab Kharadi
First supervisor Name: Irilenia Nobeli
Date: 01/06/20
version: v2.0.0'''

# importing required module
from sys import argv
import cgi
#import cgitb
import hashlib
import re
import os
#cgitb.enable()

def get_data(temp_out = 'temp.out'):
    refine_data = []
    with open (temp_out, 'r') as f:
        lines = f.read().splitlines()
        #print(lines)
        if lines:     
            seq_name = lines.pop(0)
        #print(seq_name)
            seq_name=re.sub(".fa\s\sFORWARD","",re.sub(">","",seq_name))
       
        #seq_name=re.sub(".fa\s\sFORWARD","",seq_name)
        #print(seq_name)
        #s = re.match('^(\>)(\w+\.\w+\.*)(\.fa.+)',seq_name)
        #seq_name = s.groups(0)
            data = [line.split() for line in lines]
            refine_data = [d for d in data if d[0] != 'None' and float(d[0]) < 0]
            return seq_name, refine_data

def process_fa(temp_fa, temp_out="temp.out"):
        os.system("transterm/2ndscore --no-rvs {} > {}".format(temp_fa, temp_out))
        #os.system("rm {}".format(temp_out))

def palindromes(data):
    '''This function take data from a file and return a list of palindromes.
    Each palindrome represented by a four cordinate, two for left steam and two for right steam'''
    palindrome= []
    for i in range(1,len(data)):
        word = data[i]
        energy = float(word[0])
        start = int(word[1])
        lsteam = re.sub('-','',word[5])           #Left steam of a pallindrome,cleaning data
        rsteam = re.sub('-','',word[7])           #Right steam of a pallindrome,cleaning data
        loop = word[6]
        length = int(word[3])
        p = [start,start+len(lsteam)-1,length-len(rsteam)+1,length]
        palindrome.append(p)
    return sorted(palindrome)                     # sorting list accrding to start position

In [3]:
def relation(p1,p2):
    '''This function take list of palindromes and return relationship between two palindromes in form of a Letter.
    Where O=overlapping,S=serial,X=exclusive,I=Included'''
    if p1[1] < p2[0] and p1[2] > p2[1] and p1[3] < p2[2]:
        return 'O'
    elif p1[1]<p2[0] and p1[2] > p2[3] :
        return 'I'
    elif p1[3] < p2[0]:
        return 'S'
    else:
        return 'X'

In [4]:
def pgraph(palindrome):
    '''This function give graph from each pallindrome (consider as Node) to another pallindrome.
    There is no path between pallindrome having Exclusive(X) relation.
    Rerurns dictionary which has Node as key and list of paths from that node as list'''
    #palindrome = palindromes()
    #print(len(palindrome))
    graph = {}
    for i in range(len(palindrome)):
        path = []
        for j in range(i,len(palindrome)):
            r = relation(palindrome[i],palindrome[j])
            if r != 'X':
                path.append(j)
                graph[i] = path
            else:
                graph[i] = path
    print(graph)
    return graph

In [5]:
def path(graph,start,p=[]):
    '''This function give complete paths from each pallindrome'''
    p = p + [start]
    if graph[start] == []:
        return [p]
    paths = []
    for node in graph[start]:
        if node not in p:
            new_paths = path(graph,node,p)
            for new_path in new_paths:
                paths.append(new_path)
    return paths

In [6]:
def dag(paths):
    for p in sorted(paths):
        for j in range(len(paths)-1):
            if set(p) < set(paths[j]):
                try:
                    paths.remove(p)
                    #da_graph = paths
                except:
                    next
                    continue
    return max(paths, key=len)

In [7]:
def fingerprint(da_graph,palindrome):
    '''This function provides a fingerprint for path from each node. We use hashlib to convert
    character presenting relation into a hash number'''
    paths_fingerprint = []
    i =0
    fp = ''
    while i<len(da_graph)-1:
        fp += relation(palindrome[i],palindrome[i+1])
        i += 1
    bfp = fp.encode('utf-8')                            #turn string into byte string
    hfp = hashlib.sha256(bfp)                           #turn bytes into hash object
    index= int(hfp.hexdigest(), 16) % 10**3
    paths_fingerprint.append(index)
    return paths_fingerprint

In [8]:
def tanimoto(s1, s2):
    s12 = set(s1).intersection(set(s2))
    tan = len(s12)/(len(s1) + len(s2) - len(s12))
    return tan

In [9]:
def accuracy(tan):
    if tan >= 0.8 :
        return tan80
    elif tan >= 0.6 and tan < 0.8:
        return tan60
    elif tan >= 0.4 and tan < 0.6:
        return tan40
    elif tan >= 0.2 and tan < 0.4:
        return tan20
    else:
        return tanbad

In [10]:
def run(filename=argv[1],result= {}):
    with open (filename,'r') as f:
        sequences = f.read().splitlines()
    i = 0
    while i < len(sequences):
        deal_single_seq(sequences,i)
        (seq_name, refine_data) = get_data()
        palindrome = palindromes(refine_data)
        graph = pgraph(palindrome)
        paths = path(graph,0)
        da_graph = dag(paths)
        paths_fingerprint = fingerprint(da_graph,palindrome)
        result[seq_name] = paths_fingerprint
        i += 2
    return result

In [11]:
def run_new(dir,result={}):
    for file in os.listdir(dir):
        temp_fa=f'{dir}/{file}'
        process_fa(temp_fa)
        (seq_name, refine_data) = get_data()
        print("1st and 2nd task completed: data_mining model")
        palindrome = palindromes(refine_data)
        print("palindrome function completed")
        graph = pgraph(palindrome)
        '''graph = {0:[1,2,3,7,9],
                1:[4,7],
                2:[4,5,8],
                3:[8],
                4:[],
                5:[8],
                6:[],
                7:[],
                8:[],
                9:[]}'''
        print("graph function completed")
        paths = path(graph,0)
        print("paths generated")
        da_graph = dag(paths)
        print("dag completed")
        paths_fingerprint = fingerprint(da_graph,palindrome)
        result[seq_name] = paths_fingerprint
        print("fingerprint generated")
    return result

In [12]:
path = r"C:\Users\rober\OneDrive\Desktop\MSc_Bioinformatics\Thesis\RNAid-develop\RNAid-develop\hairpins_noreverse.out"
f = open(path, 'r')
test_data = f.read().splitlines()
test_data.pop(0)
print(test_data)
my_lines = [d.split() for d in test_data]
print(my_lines)

[' 3.9       1 ..      11   xxxxxxxxxxxxxxx                -ATG AAAG CATT                 CTGGCGTAACGCCGC', ' 3.9       2 ..      12   xxxxxxxxxxxxxxA                TGAA AGCA TTC-                 TGGCGTAACGCCGCG', ' 1.4       2 ..      13   xxxxxxxxxxxxxxA                TGAA AGCA TTCT                 GGCGTAACGCCGCGT', ' 4.9       1 ..      14   xxxxxxxxxxxxxxx               ATGAA AGCA TTCTG                GCGTAACGCCGCGTT', ' 5.8       4 ..      15   xxxxxxxxxxxxATG                AAAG CATT CTGG                 CGTAACGCCGCGTTG', ' 3.5       3 ..      16   xxxxxxxxxxxxxAT               GAAAG CATT CTGGC                GTAACGCCGCGTTGC', ' 2.9       6 ..      17   xxxxxxxxxxATGAA                AGC ATTCTG GCG                 TAACGCCGCGTTGCT', '   2       5 ..      18   xxxxxxxxxxxATGA               AAGC ATTCTG GCGT                AACGCCGCGTTGCTC', ' 5.5       4 ..      19   xxxxxxxxxxxxATG              AAAGC ATTCTG GCGTA               ACGCCGCGTTGCTCG', ' 3.2      10 ..      20   xxxxxxATG

In [13]:
def get_data(temp_out):
    ## empty list
    refine_data = []
    ## opening hairpins.out file obtained
    ## from 2ndscore
    with open (temp_out, 'r') as f:
        ## creating list off each line in .out file
        lines = f.read().splitlines()
        #print(lines)
        ## extracting first line which corresponds
        ## to identifier of the sequence
        if lines:     
            seq_name = lines.pop(0)
            #print(seq_name)
        ## eliminating the fasta identifier '>'
            seq_name=re.sub(">","",seq_name)
        #seq_name=re.sub(".fa\s\sFORWARD","",seq_name)
        #print(seq_name)
        #s = re.match('^(\>)(\w+\.\w+\.*)(\.fa.+)',seq_name)
        #seq_name = s.groups(0)
        ## data separates the file by a list of lists, transforming each line in the file
        ## into a list of its own in order to better manipulate the key elements that will be used
        ## in the rest of the code, like the free energy score (first element)
            data = [line.split() for line in lines]
        ## refine data curates lines with a low free energy scores (0 or negative)
            refine_data = [d for d in data if d[0] != 'None' and float(d[0]) <= 0]
            return   seq_name, refine_data
print(get_data(path)[1])
test_refined_data = get_data(path)[1]

[['-8.2', '14', '..', '24', 'xxATGAAAGCATTCT', 'GGCG', 'TAA', 'CGCC', 'GCGTTGCTCGCGGTT'], ['-6.9', '13', '..', '25', 'xxxATGAAAGCATTC', 'TGGCG', 'TAA', 'CGCCG', 'CGTTGCTCGCGGTTT'], ['-3.4', '12', '..', '26', 'xxxxATGAAAGCATT', 'CTGGCG', 'TAA', 'CGCCGC', 'GTTGCTCGCGGTTTC'], ['-3.2', '12', '..', '27', 'xxxxATGAAAGCATT', 'C-TGGCG', 'TAA', 'CGCCGCG', 'TTGCTCGCGGTTTCT'], ['-0.6', '9', '..', '28', 'xxxxxxxATGAAAGC', 'ATTCTGGCG', 'TAA', 'CGCC-GCGT', 'TGCTCGCGGTTTCTC'], ['-5.4', '19', '..', '29', 'AAAGCATTCTGGCGT', 'AACG', 'CCG', 'CGTT', 'GCTCGCGGTTTCTCT'], ['-4.1', '18', '..', '30', 'GAAAGCATTCTGGCG', 'TAACG', 'CCG', 'CGTTG', 'CTCGCGGTTTCTCTG'], ['-6.4', '17', '..', '31', 'TGAAAGCATTCTGGC', 'GTAACG', 'CCG', 'CGTTGC', 'TCGCGGTTTCTCTGC'], ['-5', '6', '..', '32', 'xxxxxxxxxxATGAA', 'AGCATTCTGGCG', 'TAA', 'CGCCGCGTTGCT', 'CGCGGTTTCTCTGCT'], ['-5.2', '15', '..', '33', 'xATGAAAGCATTCTG', 'GCGTAACG', 'CCG', 'CGTTGCTC', 'GCGGTTTCTCTGCTT'], ['-2.4', '24', '..', '34', 'ATTCTGGCGTAACGC', 'CGCG', 'TTG', 

In [14]:
def palindromes(data):
    '''This function takes the refined data from a get_data and returns a list of palindromes.
    Each palindrome represented by a four cordinate, two for left steam and two for right steam'''
    ## creates empty list
    palindrome= []
    ## itirates through the refined data
    for i in range(0,len(data)):
        ## word divides each line in a list
        word = data[i]
        ## energy hasn't been used
        energy = float(word[0])
        ## start is the base where hairpin starts
        start = int(word[1])
        ## positions 5 and 7 tend to have -
        ## here we remove them left being 5, and right 7
        lsteam = re.sub('-','',word[5])           #Left steam of a pallindrome,cleaning data
        rsteam = re.sub('-','',word[7])           #Right steam of a pallindrome,cleaning data
        ## loop is apparently not used
        loop = word[6]
        ## index 3 represents the lenght of the hairpin
        length = int(word[3])
        ## let's figure why is this sequence of characters a palindrome
        p = [start,start+len(lsteam)-1,length-len(rsteam)+1,length]
        palindrome.append(p)
    return sorted(palindrome)                     # sorting list accrding to start position

for i in range(0,len(test_refined_data)):
    word = test_refined_data[i]
print(palindromes(test_refined_data))

[[6, 17, 21, 32], [9, 17, 21, 28], [12, 17, 21, 26], [12, 17, 21, 27], [13, 17, 21, 25], [14, 17, 21, 24], [15, 22, 26, 33], [17, 22, 26, 31], [18, 22, 26, 30], [19, 22, 26, 29], [19, 27, 33, 41], [20, 27, 33, 40], [21, 27, 33, 39], [22, 27, 33, 38], [23, 27, 33, 37], [24, 27, 31, 34], [24, 27, 33, 36], [25, 27, 33, 35], [34, 37, 43, 46], [35, 37, 43, 45]]


In [35]:
def get_data(temp_out):
    ## empty list
    refine_data = []
    ## opening hairpins.out file obtained
    ## from 2ndscore
    with open (temp_out, 'r') as f:
        ## creating list off each line in .out file
        lines = f.read().splitlines()
        #print(lines)
        ## extracting first line which corresponds
        ## to identifier of the sequence
        if lines:     
            seq_name = lines.pop(0)
            #print(seq_name)
        ## eliminating the fasta identifier '>'
            seq_name=re.sub(">","",seq_name)
        #seq_name=re.sub(".fa\s\sFORWARD","",seq_name)
        #print(seq_name)
        #s = re.match('^(\>)(\w+\.\w+\.*)(\.fa.+)',seq_name)
        #seq_name = s.groups(0)
        ## data separates the file by a list of lists, where each item is separated
        ## by the empty spaces, converting each item into a list of strings
            data = [line.split() for line in lines]
        ## refine data curates lines with a positive score
            refine_data = [d for d in data if d[0] != 'None' and float(d[0]) <= 0]
            return   seq_name, refine_data
#print(get_data('../hairpins_noreverse.out'))
refined_data = get_data('../Dataset/hairpins/sequence_1.fasta.out')
refined_data2 = get_data('../Dataset/hairpins/glutII_1.fasta.out')
refined_data3 = get_data('../Dataset/hairpins/glutII_9.fasta.out')
refined_data4 = get_data('../Dataset/hairpins/sequence_15.fasta.out')
refined_data5 = get_data('../Dataset/hairpins/sequence_8.fasta.out')
empty_file = get_data('../test_data_compempty.out')
print(refined_data[1])
#print(palindromes(refined_data))
print(empty_file)

[['-1', '33', '..', '43', 'TTAATCCGACTTTGA', 'CAAC', 'CGG', 'GATG', 'GACCTCTTACAGGTG'], ['-0.3', '38', '..', '48', 'CCGACTTTGACAACC', 'GGG', 'ATGGA', 'CCT', 'CTTACAGGTGGGAGA'], ['-2.5', '46', '..', '56', 'GACAACCGGGATGGA', 'CCT', 'CTTAC', 'AGG', 'TGGGAGAGACGGAAG'], ['-3.4', '45', '..', '57', 'TGACAACCGGGATGG', 'ACCT', 'CTTAC', 'AGGT', 'GGGAGAGACGGAAGT'], ['-0.1', '46', '..', '59', 'GACAACCGGGATGGA', 'CCTCT', 'TACA', 'GGTGG', 'GAGAGACGGAAGTAA'], ['-1.8', '47', '..', '62', 'ACAACCGGGATGGAC', 'CTCTTA', 'CAGG', 'TGGGAG', 'AGACGGAAGTAAGGA'], ['-1.9', '47', '..', '64', 'ACAACCGGGATGGAC', 'CTCTTAC', 'AGGT', 'GGGAGAG', 'ACGGAAGTAAGGATA'], ['-5.4', '74', '..', '84', 'GGAGAGACGGAAGTA', 'AGGA', 'TAA', 'TCCT', 'GAAGGAACGCGCCxx'], ['-1.9', '73', '..', '85', 'GGGAGAGACGGAAGT', 'AAGGA', 'TAA', 'TCCTG', 'AAGGAACGCGCCxxx'], ['-2.8', '72', '..', '86', 'TGGGAGAGACGGAAG', 'TAAGGA', 'TAA', 'TCCTGA', 'AGGAACGCGCCxxxx'], ['0', '80', '..', '91', 'ACGGAAGTAAGGATA', 'ATCC', 'TGAA', 'GGAA', 'CGCGCCxxxxxxxxx']]
N

In [16]:
def palindromes(data):
    '''This function takes the refined data from a get_data and returns a list of palindromes.
    Each palindrome represented by a four cordinate, two for left steam and two for right steam'''
    ## creates empty list
    palindrome= []
    ## itirates through the refined data
    for i in range(0,len(data)):
        ## word divides each line in a list
        word = data[i]
        ## energy hasn't been used
        energy = float(word[0])
        ## start is the base where hairpin starts
        start = int(word[1])
        ## positions 5 and 7 tend to have -
        ## here we remove them left being 5, and right 7
        lsteam = re.sub('-','',word[5])           #Left steam of a pallindrome,cleaning data
        rsteam = re.sub('-','',word[7])           #Right steam of a pallindrome,cleaning data
        ## loop is apparently not used
        loop = word[6]
        ## index 3 represents the lenght of the hairpin
        length = int(word[3])
        ## let's figure why is this sequence of characters a palindrome
        p = [start,start+len(lsteam)-1,length-len(rsteam)+1,length]
        palindrome.append(p)
    return sorted(palindrome)                     # sorting list accrding to start position
print(palindromes(refined_data[1]))
palindrome = palindromes(refined_data[1])
graph_data = palindromes(refined_data[1])
graph_data2 = palindromes(refined_data2[1])
graph_data3 = palindromes(refined_data3[1])
graph_data4 = palindromes(refined_data4[1])
graph_data5 = palindromes(refined_data5[1])
print(refined_data[1])

[[33, 36, 40, 43], [38, 40, 46, 48], [45, 48, 54, 57], [46, 48, 54, 56], [46, 50, 55, 59], [47, 52, 57, 62], [47, 53, 58, 64], [72, 77, 81, 86], [73, 77, 81, 85], [74, 77, 81, 84], [80, 83, 88, 91]]
[['-1', '33', '..', '43', 'TTAATCCGACTTTGA', 'CAAC', 'CGG', 'GATG', 'GACCTCTTACAGGTG'], ['-0.3', '38', '..', '48', 'CCGACTTTGACAACC', 'GGG', 'ATGGA', 'CCT', 'CTTACAGGTGGGAGA'], ['-2.5', '46', '..', '56', 'GACAACCGGGATGGA', 'CCT', 'CTTAC', 'AGG', 'TGGGAGAGACGGAAG'], ['-3.4', '45', '..', '57', 'TGACAACCGGGATGG', 'ACCT', 'CTTAC', 'AGGT', 'GGGAGAGACGGAAGT'], ['-0.1', '46', '..', '59', 'GACAACCGGGATGGA', 'CCTCT', 'TACA', 'GGTGG', 'GAGAGACGGAAGTAA'], ['-1.8', '47', '..', '62', 'ACAACCGGGATGGAC', 'CTCTTA', 'CAGG', 'TGGGAG', 'AGACGGAAGTAAGGA'], ['-1.9', '47', '..', '64', 'ACAACCGGGATGGAC', 'CTCTTAC', 'AGGT', 'GGGAGAG', 'ACGGAAGTAAGGATA'], ['-5.4', '74', '..', '84', 'GGAGAGACGGAAGTA', 'AGGA', 'TAA', 'TCCT', 'GAAGGAACGCGCCxx'], ['-1.9', '73', '..', '85', 'GGGAGAGACGGAAGT', 'AAGGA', 'TAA', 'TCCTG', 'A

In [17]:
def relation(p1,p2):
    '''This function takes the list of palindromes and return relationship between two palindromes in form of a Letter.
    Where O=overlapping,S=serial,X=exclusive,I=Included'''
    if p1[1] < p2[0] and p1[2] > p2[1] and p1[3] < p2[2]:
        return 'O'
    elif p1[1]<p2[0] and p1[2] > p2[3] :
        return 'I'
    elif p1[3] < p2[0]:
        return 'S'
    else:
        return 'X'

def pgraph(palindrome):
    '''This function give graph from each palindrome (consider as Node) to another palindrome.
    There is no path between palindrome having Exclusive(X) relation.
    Rerurns dictionary which has Node as key and list of paths from that node as list'''
    #palindrome = palindromes()
    #print(len(palindrome))
    graph = {}
    ## itirates through each element in the palindromes list
    ## note that palindromes is a list of lists
    for i in range(len(palindrome)):
        ## empty list for each list in palindrome
        path = []
        ## itirates through palindome iteration
        for j in range(i,len(palindrome)):
            r = relation(palindrome[i],palindrome[j])
            if r != 'X':
                path.append(j)
                graph[i] = path
            else:
                graph[i] = path
    return graph
test_graph = pgraph(graph_data)
test_graph2 = pgraph(graph_data2)
test_graph3 = pgraph(graph_data3)
test_graph4 = pgraph(graph_data4)
test_graph5 = pgraph(graph_data5)
print(test_graph)
print(test_graph2)
print(test_graph3)
# print(graph_data)
# print(len(test_graph), len(graph_data))

{0: [2, 3, 4, 5, 6, 7, 8, 9, 10], 1: [7, 8, 9, 10], 2: [7, 8, 9, 10], 3: [7, 8, 9, 10], 4: [7, 8, 9, 10], 5: [7, 8, 9, 10], 6: [7, 8, 9, 10], 7: [], 8: [], 9: [], 10: []}
{0: [4, 5, 6], 1: [4, 5, 6], 2: [4, 5, 6], 3: [4, 5, 6], 4: [6], 5: [], 6: []}
{0: [14, 15, 16], 1: [14, 15, 16], 2: [13, 14, 15, 16], 3: [13, 14, 15, 16], 4: [11, 12, 13, 14, 15, 16], 5: [11, 12, 13, 14, 15, 16], 6: [11, 12, 13, 14, 15, 16], 7: [11, 12, 13, 14, 15, 16], 8: [11, 12, 13, 14, 15, 16], 9: [11, 12, 13, 14, 15, 16], 10: [11, 12, 13, 14, 15, 16], 11: [16], 12: [], 13: [], 14: [], 15: [], 16: []}


In [18]:
def path(graph,start,p=[]):
    '''This function give complete paths from each pallindrome'''
    p = p + [start]
    if graph[start] == []:
        return [p]
    paths = []
    for node in graph[start]:
        if node not in p:
            new_paths = path(graph,node,p)
            for new_path in new_paths:
                paths.append(new_path)
    return paths
test_path = path(test_graph, 0)
test_path2 = path(test_graph2, 0)
test_path3 = path(test_graph3, 0)
test_path4 = path(test_graph4, 0)
test_path5 = path(test_graph5, 0)
print(test_path)
print(test_path2)
print(test_path3)

[[0, 2, 7], [0, 2, 8], [0, 2, 9], [0, 2, 10], [0, 3, 7], [0, 3, 8], [0, 3, 9], [0, 3, 10], [0, 4, 7], [0, 4, 8], [0, 4, 9], [0, 4, 10], [0, 5, 7], [0, 5, 8], [0, 5, 9], [0, 5, 10], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 6, 10], [0, 7], [0, 8], [0, 9], [0, 10]]
[[0, 4, 6], [0, 5], [0, 6]]
[[0, 14], [0, 15], [0, 16]]


In [19]:
def dag(paths):
    for p in sorted(paths):
        for j in range(len(paths)-1):
            if set(p) < set(paths[j]):
                try:
                    paths.remove(p)
                    #da_graph = paths
                except:
                    next
                    continue
    return max(paths, key=len)
dagged_data = dag(test_path)
dagged_data2 = dag(test_path2)
dagged_data3 = dag(test_path3)
dagged_data4 = dag(test_path4)
dagged_data5 = dag(test_path5)
print(dagged_data)
print(dagged_data2)
print(dagged_data3)

[0, 2, 7]
[0, 4, 6]
[0, 14]


In [20]:
def fingerprint(da_graph,palindrome):
    '''This function provides a fingerprint for path from each node. We use hashlib to convert
    character presenting relation into a hash number'''
    paths_fingerprint = []
    i =0
    fp = ''
    while i<len(da_graph)-1:
        fp += relation(palindrome[i],palindrome[i+1])
        i += 1
    bfp = fp.encode('utf-8')                            #turn string into byte string
    hfp = hashlib.sha256(bfp)                           #turn bytes into hash object
    index= int(hfp.hexdigest(), 16) % 10**3
    paths_fingerprint.append(index)
    return paths_fingerprint
fingerprints = fingerprint(dagged_data, graph_data)
fingerprints2 = fingerprint(dagged_data2, graph_data2)
fingerprints3 = fingerprint(dagged_data3, graph_data3)
fingerprints4 = fingerprint(dagged_data4, graph_data4)
fingerprints5 = fingerprint(dagged_data5, graph_data5)
print(fingerprints, fingerprints2, fingerprints3, fingerprints4, fingerprints5)

[168] [168] [741] [168] [741]


In [21]:
simple_data = open("../hairpins_noreverse.out")

In [22]:
print(simple_data.read())

>NC_000913.3:c645003-644197 rna [organism=Escherichia coli str. K-12 substr. MG1655] [GeneID=949065] [chromosome=] FORWARD
 3.9       1 ..      11   xxxxxxxxxxxxxxx                -ATG AAAG CATT                 CTGGCGTAACGCCGC
 3.9       2 ..      12   xxxxxxxxxxxxxxA                TGAA AGCA TTC-                 TGGCGTAACGCCGCG
 1.4       2 ..      13   xxxxxxxxxxxxxxA                TGAA AGCA TTCT                 GGCGTAACGCCGCGT
 4.9       1 ..      14   xxxxxxxxxxxxxxx               ATGAA AGCA TTCTG                GCGTAACGCCGCGTT
 5.8       4 ..      15   xxxxxxxxxxxxATG                AAAG CATT CTGG                 CGTAACGCCGCGTTG
 3.5       3 ..      16   xxxxxxxxxxxxxAT               GAAAG CATT CTGGC                GTAACGCCGCGTTGC
 2.9       6 ..      17   xxxxxxxxxxATGAA                AGC ATTCTG GCG                 TAACGCCGCGTTGCT
   2       5 ..      18   xxxxxxxxxxxATGA               AAGC ATTCTG GCGT                AACGCCGCGTTGCTC
 5.5       4 ..      19   xxxxxxxxxxxxATG    

In [23]:
# shuffle sequences from rfam family
# to introduce randomness 

In [24]:
## creation of fake families, get a distribution of how non-meaninful data would look like.
## 

In [25]:
print(type(range(10)))

<class 'range'>


In [26]:
a = [12, 17, 21, 26]
b = [14, 17, 21, 24]
relation(a, b)
c = [a, b]
c

[[12, 17, 21, 26], [14, 17, 21, 24]]

In [27]:
pgraph(c)

{0: [], 1: []}

In [28]:
fingerprint(dag(path(pgraph(c), 0)), c)

[549]

In [29]:
set([0,1,2])<set([10])

False

In [30]:
len(set([0,2,7]))

3

In [31]:
{0, 2, 7}>={0,2}

True

In [32]:
{'C', 'F'} > {'C', 'F'}

False

In [33]:
set([0,2,7]) <= set([0,2])

False

In [34]:
set([7]) <= set([0,2,7])

True